# Map version update 2023

In [1]:
import configparser
from pathlib import Path
import pandas as pd
import pandas.io.sql as pandasql
from psycopg2 import connect
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator, PercentFormatter)
import numpy
from psycopg2.extras import execute_values
from datetime import datetime
import matplotlib as mpl
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from IPython.display import HTML
import matplotlib.patches as patches
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('db.cfg')))
from itertools import combinations, chain
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

/etc/jupyterhub/.venv/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


## Find out how many link_dir changed in the new map version

2329 links changed in the new map version.

In [5]:
num_links = '''
WITH temp AS (
select link_dir from congestion.network_links_22_2
except 
select link_dir from here.routing_streets_23_1 )
SELECT count(1) from temp
'''
num_links = pandasql.read_sql(num_links, con)

num_links

,count
0,2329


## Find out how many segment_id needs to be updated

178 segments in the congestion network needs to be updated.

In [6]:
num_links = '''
with changed_links AS (
	select link_dir from congestion.network_links_21_1
	except 
	select link_dir from here.routing_streets_22_2)
	
select count(distinct segment_id) from congestion.network_links_21_1 
inner join changed_links using (link_dir)
'''
num_links = pandasql.read_sql(num_links, con)

num_links

,count
0,178


## Find out if any segments are affected by node_id changes

Nothing! network nodes stay the same this time as well.

In [8]:

num_links = '''
select * from congestion.network_segments
inner join (select node_id from congestion.network_nodes
			except 
			select node_id from here.routing_nodes_23_1 )a on start_vid = node_id or end_vid = node_id
'''
num_links = pandasql.read_sql(num_links, con)

num_links

,segment_id,start_vid,end_vid,geom,total_length,highway,direction,node_id


## Find out how many new traffic signals have been added